In [265]:
from external_potential import Framework, PotentialType
from dft_ad_jax import *
from si_units import *
from glob import glob
import numpy as np
from feos.dft import *
from feos.pcsaft import PcSaftParameters as PcSaftParametersFeos
import jax.numpy as jnp
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import traceback
import os
import csv
from datetime import datetime
import json

In [266]:
def generate_framework(cif_file):#, rc, ngrid, solid_parameters, fluid_parameters):
    return Framework(
        cif=cif_file,
        #cutoff_radius=rc,
        #ngrid=ngrid,
        #solid_parameters=solid_parameters,
        #fluid_parameters=fluid_parameters,
        #maximum_reduced_energy=10,
        #jax_enable_x64=True
    )

In [267]:
def vext_features(framework, solid_parameters, fluid_parameters, temperature, bins=bins):
    vext_pot = framework.calculate_external_potential(
        ngrid=(32,)*3, 
        interaction_potential_type=PotentialType.LennardJones, 
        cutoff_radius=12.0, 
        fluid_parameters=fluid_parameters, 
        solid_parameters=solid_parameters, 
        #binary_interaction_parameters=binary, 
        #tail_correction_type='RASPA', 
        #block_files=['block1.res', 'block2.res'],
    )   
    #print(vext_pot.external_potential_k)
    features, edges = jnp.histogram(vext_pot.external_potential_k/temperature, bins=bins, range=(-15,10))
    return features, framework.unitcell.volume, vext_pot, framework, edges

In [268]:
def calc_dft_adsorption(
        dft: DFT,
        state: State,
        solver: dict,
        grid: Grid,
        vext_pot,
        temperature,
        initial_dens_profile=None,
):
    # externe Potential vorbereiten
    ext = vext_pot.external_potential_k / temperature
    ext = jnp.asarray(ext, dtype=jnp.float32)
    ext = jnp.clip(ext, -10.0, 10.0)

    # Bulk Dichte
    density_bulk = jnp.array(state.partial_density * NAV * ANGSTROM**3)
    density_bulk_dim = density_bulk[:, None, None, None] * jnp.ones([len(components)] + grid.n_grid)

    # Startprofil
    rho_init = initial_dens_profile if initial_dens_profile is not None else density_bulk_dim
    rho_init = jnp.clip(rho_init, 1e-12, None)

    # verschiedene Dämpfungen probieren
    damping_list = [
        (solver["damp_coeff_1"], solver["damp_coeff_2"]),  # zuerst Standard
        #(0.2, 0.1),
        #(0.01, 0.005),
        #(0.01, 0.005),
        #(0.005, 0.002),
        (0.002, 0.001),
        (0.001, 0.0005),
        (0.0001, 0.00005),
        (0.00001, 0.000005),
        (0.000001, 0.0000005),
    ]

    for d1, d2 in damping_list:
        try:
            density_eq, _, _ = dft.picard_iteration_log(
                rho_init,
                density_bulk_dim,
                dft.df_drho(density_bulk_dim)[:, 0, 0, 0],
                ext,
                max_iter=solver["max_iter_1"],
                damping_constant=d1,
                tol=solver["tol"]
            )
            density_eq, _, _ = dft.picard_iteration_log(
                density_eq,
                density_bulk_dim,
                dft.df_drho(density_bulk_dim)[:, 0, 0, 0],
                ext,
                max_iter=solver["max_iter_2"],
                damping_constant=d2,
                tol=solver["tol"]
            )
            print(f"Konvergenz erreicht mit damping ({d1}, {d2})")
            return density_eq, dft.df_drho(density_bulk_dim)[:, 0, 0, 0], density_bulk

        except Exception:
            print(f"damping ({d1}, {d2}) fehlgeschlagen, versuche weiter...")

    raise RuntimeError("Keine Konvergenz trotz Dämpfungsversuchen.")

In [269]:
cif_files = glob("/Users/danielbock/MOF_CIF/*.cif")
failed_structures = []

pressures=[1*BAR] #np.linspace(0.1, 100, 5) * BAR
temperatures=[400*KELVIN] #np.linspace(300, 500, 5) * KELVIN
ngrid = 32
bins=100

# Hier wieder korrigieren
solid_parameters = {
    "O": { "sigma": 3.21, "epsilon_k": 76.3, "k_si": 0.0},
    "Si": { "sigma": 3.3, "epsilon_k": 22.0, "k_si": 0.0 },
}

components = ['methane']
fluid_parameters = {
    "methane": {"m": 1.0, "sigma": 3.70051, "epsilon_k": 150.07147},
}
fluid_file = 'parameters/pcsaft/esper2023.json'
pcsaft_params = pcsaft_from_file(components, fluid_file)
params_feos = PcSaftParametersFeos.from_json(components, fluid_file)
func = HelmholtzEnergyFunctional.pcsaft(params_feos, FMTVersion.AntiSymWhiteBear)
rc = 12.0

base_solver = {
    "damp_coeff_1": 0.01,   # vorher: "damping_coefficient_1"
    "damp_coeff_2": 0.005,   # vorher: "damping_coefficient_2"
    "max_iter_1": 500,
    "max_iter_2": 1000,
    "tol": 1e-4,
}


In [264]:
"""cif_file = "/Users/danielbock/MOF_CIF/2008[ZnSi][nan]3[ASR]1.cif"
temperature=400*KELVIN
pressure=1*BAR

framework = generate_framework(cif_file)#, rc, ngrid, solid_parameters, fluid_parameters)
grid = Grid(
    n_grid=(ngrid, ngrid, ngrid),  # 3D Grid definieren
    length=framework.unitcell.lattice.lengths,
    skew_angles=framework.unitcell.lattice.angles,
    lanczos_sigma=True
)
#func = HelmholtzEnergyFunctional.pcsaft(param_feos, FMTVersion.AntiSymWhiteBear)
features, volume, vext_pot, framework, edges = vext_features(framework, solid_parameters, fluid_parameters, float(temperature / KELVIN), bins=bins)
grid.n_grid = list(grid.n_grid)
dft  = DFT(grid, pcsaft_params, temperature / KELVIN)
state = State(func, temperature=temperature, pressure=pressure)
density_eq, chemical_potential_res_bulk, density_bulk = calc_dft_adsorption(dft, state, base_solver, grid, vext_pot, temperature/KELVIN)
"""

'cif_file = "/Users/danielbock/MOF_CIF/2008[ZnSi][nan]3[ASR]1.cif"\ntemperature=400*KELVIN\npressure=1*BAR\n\nframework = generate_framework(cif_file)#, rc, ngrid, solid_parameters, fluid_parameters)\ngrid = Grid(\n    n_grid=(ngrid, ngrid, ngrid),  # 3D Grid definieren\n    length=framework.unitcell.lattice.lengths,\n    skew_angles=framework.unitcell.lattice.angles,\n    lanczos_sigma=True\n)\n#func = HelmholtzEnergyFunctional.pcsaft(param_feos, FMTVersion.AntiSymWhiteBear)\nfeatures, volume, vext_pot, framework, edges = vext_features(framework, solid_parameters, fluid_parameters, float(temperature / KELVIN), bins=bins)\ngrid.n_grid = list(grid.n_grid)\ndft  = DFT(grid, pcsaft_params, temperature / KELVIN)\nstate = State(func, temperature=temperature, pressure=pressure)\ndensity_eq, chemical_potential_res_bulk, density_bulk = calc_dft_adsorption(dft, state, base_solver, grid, vext_pot, temperature/KELVIN)\n'

In [261]:
if not os.path.exists("features_results.csv"):
    with open("features_results.csv", mode="w", newline="") as f:
        writer = csv.writer(f)
        # Dynamische Spalten für Bins (Histogrammwerte)
        feature_headers = [f"bin_{i}" for i in range(bins)]
        writer.writerow(["structure_name", "temperature_K"] + feature_headers)
        
if not os.path.exists("dft_results.csv"):
    with open("dft_results.csv", mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "structure_name", "temperature_K", "pressure_bar",
            "volume", "ngrid", "loading_atoms", "loading_mol_kg",
            "density_bulk", "chem_pot_res_bulk", "voxel_volume"
        ])

In [262]:
# Schleifen
for cif_file in tqdm(cif_files):
    structure_name = os.path.basename(cif_file).replace(".cif", "")
    framework = generate_framework(cif_file)#, rc, ngrid, solid_parameters, fluid_parameters)
    grid = Grid(
        n_grid=(ngrid, ngrid, ngrid),  # 3D Grid definieren
        length=framework.unitcell.lattice.lengths,
        skew_angles=framework.unitcell.lattice.angles,
        lanczos_sigma=True
    )
    
    for temperature in temperatures:
        #func = HelmholtzEnergyFunctional.pcsaft(param_feos, FMTVersion.AntiSymWhiteBear)
        features, volume, vext_pot, framework, edges = vext_features(framework, solid_parameters, fluid_parameters, float(temperature / KELVIN), bins=bins)
        grid.n_grid = list(grid.n_grid)

        with open("features_results.csv", mode="a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([structure_name, temperature] + list(features))
            
        dft  = DFT(grid, pcsaft_params, temperature / KELVIN)
        
        for pressure in pressures:
            state = State(func, temperature=temperature, pressure=pressure)
            density_eq, chemical_potential_res_bulk, density_bulk = calc_dft_adsorption(dft, state, base_solver, grid, vext_pot, temperature/KELVIN)
            
            loading_atoms = density_eq.sum()*grid.dv
            loading_mol_kg = loading_atoms * 1337  # ggf. anpassen

            with open("dft_results.csv", mode="a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([
                    structure_name, temperature, pressure,
                    volume, ngrid, loading_atoms,
                    loading_mol_kg, density_bulk,
                    chemical_potential_res_bulk, grid.dv
                ])

 24%|███████████████▉                                                  | 121/500 [00:00<00:01, 288.96it/s]


damping (0.01, 0.005) fehlgeschlagen, versuche weiter...



100%|██████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 274.67it/s]

  4%|██▉                                                               | 44/1000 [00:00<00:03, 271.61it/s]


damping (0.002, 0.001) fehlgeschlagen, versuche weiter...



100%|██████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 288.68it/s]

  0%|                                                                  | 1/2664 [00:08<6:06:52,  8.27s/it]

...

Konvergenz erreicht mit damping (0.0001, 5e-05)



 18%|███████████▊                                                       | 88/500 [00:00<00:01, 287.22it/s]


damping (0.01, 0.005) fehlgeschlagen, versuche weiter...



  4%|██▋                                                             | 113/2664 [23:52<8:58:48, 12.67s/it]


KeyboardInterrupt: 